<a href="https://colab.research.google.com/github/benjaminnigjeh/keyProteoforms/blob/main/quantitativeDiscovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import extrenal libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#Import databank

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_hdf("/content/drive/MyDrive/databank_updated", key="databank")

#Generate extracted ion chromatograms

In [ ]:
df = df[df["scan_type"] == "MS1"]
df = df[df["sample_prep"] == "Pellet"]
df = df[df["group_name"] == "Normal Healthy"]

extracted_ions = []

for i in range(0, 40, 1):
    filtered_df = df[(df["retntion time"] >= i) & (df["retntion time"] <= i + 1)]
    element_wise_sum = [sum(x) for x in zip(*filtered_df["cast spectra"])]
    extracted_ions.append(element_wise_sum)

#Helper functions

In [ ]:
def profile(extracted_ions, mass):
    n = int((mass-600)*10)
    addition = []
    for i in range(0, len(extracted_ions)):
        start = max(n - 5, 0)  # Ensure we don't go below index 0
        end = min(n + 5 + 1, len(extracted_ions[i]))  # Ensure we don't exceed the list length
        neighbors_sum = sum(extracted_ions[i][start:n]) + sum(extracted_ions[i][n:end])
        addition.append(neighbors_sum)
    return(addition)


def plot_multiple_y(x_values, y_values_list, labels=None):
    plt.figure(figsize=(8, 5))

    # If no labels are provided, generate default labels
    if labels is None:
        labels = [f"Series {i+1}" for i in range(len(y_values_list))]

    # Plot each y-values list
    for y_values, label in zip(y_values_list, labels):
        plt.plot(x_values, y_values, marker="o", linestyle="-", label=label)

    # Labels and title
    plt.xlabel("Retention time / minute")
    plt.ylabel("Intensity")
    plt.title("")
    plt.ylim((0,2.2e9))
    plt.legend()
    plt.grid(True)

    # Show plot
    plt.show()



#Generate charge state profiles

In [ ]:
x_values = list(range(1, 41))  # X values from 1 to 40
y1 = profile(extracted_ions, 813.3)  # First series
y2 = profile(extracted_ions,861)  # Second series
y3 = profile(extracted_ions,915)  # Third series
y4 = profile(extracted_ions,975.8)  # Fourth series
y5 = profile(extracted_ions,1045.6)  # Fifth series
y6 = profile(extracted_ions,1125.9)  # Sixth series

# Call the function with multiple Y lists
plot_multiple_y(x_values, [y1, y2, y3, y4, y5, y6], labels=[24, 23, 21, 10, 19, 17])
